## Notebook for test that model endpoints inherit tags and labels from model artifact

#### Create a project

In [ ]:
import mlrun

In [ ]:
project = mlrun.get_or_create_project("testing-model-endpoint")

In [ ]:
project.set_model_monitoring_credentials(endpoint_store_connection="v3io", stream_path="v3io", tsdb_connection="v3io")

In [ ]:
# Clean up monitoring data
project.disable_model_monitoring(delete_stream_function=True)

In [ ]:
project.enable_model_monitoring(wait_for_deployment=True)

### Train and deploy a model with tag==v1 and label test=1

In [ ]:
%%writefile train.py
from sklearn.datasets import make_classification
from mlrun.frameworks.sklearn import apply_mlrun
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import pandas as pd

def train(context,i):
    X,y = make_classification(n_samples=100,n_features=10,random_state=i)
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=i)
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train,columns=["label"])
    training_set = pd.concat([X_train,y_train],axis=1)
    model = LinearRegression()
    model.fit(X_train,y_train)
    context.log_model(key=f"model",body=pickle.dumps(model),model_file=f'model_{i}.pkl',training_set=training_set,label_column="label",tag=f"v{i}",labels={"test":i})


In [ ]:
project.set_function(func='train.py',kind='job',image="mlrun/mlrun",handler='train',name='train')

In [ ]:
i=1

In [ ]:
run = project.run_function('train',params={"i":i})

In [ ]:
serving = mlrun.new_function('serving-model',kind='serving',image='mlrun/mlrun')

In [ ]:
graph = serving.set_topology("flow", engine="sync")
router = graph.add_step("*mlrun.serving.ModelRouter", name="model-routing")

In [ ]:
models_uri = [model.uri for model in project.list_models(tag=f"v{i}")]
len(models_uri)

In [ ]:
from tqdm import tqdm
for uri in tqdm(models_uri):
    router.add_route(key=f'model',model_path=uri,class_name='mlrun.frameworks.sklearn.SKLearnModelServer')

In [ ]:
serving.set_tracking()

In [ ]:
project.deploy_function(serving)

In [ ]:
inputs = [[-0.13554955389680073,0.0076506575048474225,0.6287761723991921,-0.8751269647375463,-1.0660002219502747,-1.449969473671631,-1.1047354035566603,0.20042964461755708,0.6400321379372533,0.0785128175305061,-0.16572271960124885,0.8553736325613692,-0.4950274467481983,0.3036076443871318,-0.9177524223010839,0.3994151051172195,-0.2588778273252802,0.7705348945630903,0.14223688502019674,-0.722042747480339][:10]]

In [ ]:
serving.invoke(f"/v2/models/model/infer", {"inputs": inputs})

In [ ]:
db = mlrun.get_run_db()

In [ ]:
end_point = db.list_model_endpoints(project=project.name,model=f"model:v{i}")[0]

#### Test model tag and label

In [ ]:
#test model endpoint labels
assert end_point.metadata.labels == {'test': i}

In [ ]:
#test model endpoint tag
assert end_point.spec.model.split(":")[-1]==f"v{i}"

### Train and deploy a model with tag==v2 and label test=2

In [ ]:
%%writefile train.py
from sklearn.datasets import make_classification
from mlrun.frameworks.sklearn import apply_mlrun
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import pandas as pd

def train(context,i):
    X,y = make_classification(n_samples=100,n_features=10,random_state=i)
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=i)
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train,columns=["label"])
    training_set = pd.concat([X_train,y_train],axis=1)
    model = LinearRegression()
    model.fit(X_train,y_train)
    context.log_model(key=f"model",body=pickle.dumps(model),model_file=f'model_{i}.pkl',training_set=training_set,label_column="label",tag=f"v{i}",labels={"test":i})


In [ ]:
project.set_function(func='train.py',kind='job',image="mlrun/mlrun",handler='train',name='train')

In [ ]:
i=2

In [ ]:
run = project.run_function('train',params={"i":i})

In [ ]:
serving = mlrun.new_function('serving-model',kind='serving',image='mlrun/mlrun')

In [ ]:
graph = serving.set_topology("flow", engine="sync")
router = graph.add_step("*mlrun.serving.ModelRouter", name="model-routing")

In [ ]:
models_uri = [model.uri for model in project.list_models(tag=f"v{i}")]
len(models_uri)

In [ ]:
assert "v2" in models_uri[0]

In [ ]:
from tqdm import tqdm
for uri in tqdm(models_uri):
    router.add_route(key=f'model',model_path=uri,class_name='mlrun.frameworks.sklearn.SKLearnModelServer')

In [ ]:
serving.set_tracking()

In [ ]:
project.deploy_function(serving)

In [ ]:
inputs = [[-0.13554955389680073,0.0076506575048474225,0.6287761723991921,-0.8751269647375463,-1.0660002219502747,-1.449969473671631,-1.1047354035566603,0.20042964461755708,0.6400321379372533,0.0785128175305061,-0.16572271960124885,0.8553736325613692,-0.4950274467481983,0.3036076443871318,-0.9177524223010839,0.3994151051172195,-0.2588778273252802,0.7705348945630903,0.14223688502019674,-0.722042747480339][:10]]

In [ ]:
serving.invoke(f"/v2/models/model/infer", {"inputs": inputs})

In [ ]:
db = mlrun.get_run_db()

In [ ]:
end_point = db.list_model_endpoints(project=project.name,model=f"model:v{i}")[0]

#### Test model tag and label

In [ ]:
# test model endpoint labels
assert end_point.metadata.labels == {'test': i}

In [ ]:
# test model endpoint tags
assert end_point.spec.model.split(":")[-1]==f"v{i}"

In [ ]:
db.delete_project(name=project.name, deletion_strategy="cascade")